<a href="https://colab.research.google.com/github/Charly190605/trabajosPOO/blob/main/GestorCumplea%C3%B1os_HurtadoLinaresJuanCarlos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install -q streamlit

In [45]:
%%writefile birthday.py
import csv
import random
from datetime import datetime, date
import smtplib
from email.mime.text import MIMEText

class Birthday:
    def __init__(self, name, birthdate, email, message_id=None):
        self.name = name
        self.birthdate = datetime.strptime(birthdate, "%Y-%m-%d").date()
        self.email = email
        self.message_id = message_id  # id del mensaje personalizado o None

    def days_until_birthday(self):
        today = date.today()
        next_birthday = self.birthdate.replace(year=today.year)
        if next_birthday < today:
            next_birthday = next_birthday.replace(year=today.year + 1)
        return (next_birthday - today).days

class BirthdayManager:
    def __init__(self, filename="birthdays.csv"):
        self.filename = filename
        self.birthdays = self.load_birthdays()

    def load_birthdays(self):
        birthdays = []
        try:
            with open(self.filename, newline='', encoding="utf-8") as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    message_id = int(row["message_id"]) if row.get("message_id") else None
                    birthdays.append(Birthday(row["name"], row["birthdate"], row["email"], message_id))
        except FileNotFoundError:
            pass
        return birthdays

    def save_birthdays(self):
        with open(self.filename, "w", newline='', encoding="utf-8") as csvfile:
            fieldnames = ["name", "birthdate", "email", "message_id"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for b in self.birthdays:
                writer.writerow({
                    "name": b.name,
                    "birthdate": b.birthdate.strftime("%Y-%m-%d"),
                    "email": b.email,
                    "message_id": b.message_id if b.message_id is not None else ""
                })

    def add_birthday(self, name, birthdate, email, message_id=None):
        self.birthdays.append(Birthday(name, birthdate, email, message_id))
        self.save_birthdays()

    def get_upcoming_birthdays(self):
        return sorted(self.birthdays, key=lambda b: b.days_until_birthday())

class Message:
    def __init__(self, id_, text):
        self.id = id_
        self.text = text

class MessageManager:
    def __init__(self, filename="messages.csv"):
        self.filename = filename
        self.messages = self.load_messages()

    def load_messages(self):
        messages = []
        try:
            with open(self.filename, newline='', encoding="utf-8") as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    messages.append(Message(int(row["id"]), row["text"]))
        except FileNotFoundError:
            pass
        return messages

    def save_messages(self):
        with open(self.filename, "w", newline='', encoding="utf-8") as csvfile:
            fieldnames = ["id", "text"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for m in self.messages:
                writer.writerow({"id": m.id, "text": m.text})

    def add_message(self, text):
        new_id = max([m.id for m in self.messages], default=0) + 1
        self.messages.append(Message(new_id, text))
        self.save_messages()

    def get_random_message(self):
        if not self.messages:
            return "¡Feliz cumpleaños!"
        return random.choice(self.messages).text

    def get_message_by_id(self, id_):
        for m in self.messages:
            if m.id == id_:
                return m.text
        return None

    def delete_message(self, id_):
        self.messages = [m for m in self.messages if m.id != id_]
        self.save_messages()

    def update_message(self, id_, new_text):
        for m in self.messages:
            if m.id == id_:
                m.text = new_text
                break
        self.save_messages()

def send_email(to_email, subject, body, from_email, from_password, smtp_server="smtp.gmail.com", smtp_port=587):
    import ssl
    message = f"""\
Subject: {subject}

{body}
"""
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(from_email, from_password)
        server.sendmail(from_email, to_email, message.encode("utf-8"))


def enviar_felicitaciones(birthday_manager, message_manager, from_email, from_password):
    hoy = date.today()
    for b in birthday_manager.birthdays:
        if b.birthdate.month == hoy.month and b.birthdate.day == hoy.day:
            if b.message_id is not None:
                texto = message_manager.get_message_by_id(b.message_id) or message_manager.get_random_message()
            else:
                texto = message_manager.get_random_message()
            texto_formateado = texto.format(name=b.name)

            try:
                send_email(b.email, "¡Feliz Cumpleaños!", texto_formateado, from_email, from_password)
                print(f"Correo enviado a {b.name} ({b.email})")
            except Exception as e:
                print(f"Error enviando correo a {b.name}: {e}")


Overwriting birthday.py


In [46]:
%%writefile app.py

import streamlit as st
from birthday import BirthdayManager, MessageManager, enviar_felicitaciones

bm = BirthdayManager()
mm = MessageManager()

st.title("🎂 Gestor de Cumpleaños")

menu = ["Registrar Cumpleaños", "Ver Próximos Cumpleaños", "Gestionar Mensajes", "Enviar Felicitaciones Hoy"]
choice = st.sidebar.selectbox("Menú", menu)

if choice == "Registrar Cumpleaños":
    st.subheader("Añadir Nuevo Cumpleaños")
    name = st.text_input("Nombre")
    birthdate = st.date_input("Fecha de nacimiento")
    email = st.text_input("Correo electrónico")

    options = ["-- Ninguno --"] + [f"{m.id}: {m.text[:30]}..." for m in mm.messages]
    selected = st.selectbox("Selecciona mensaje personalizado (opcional)", options)
    selected_id = None
    if selected != "-- Ninguno --":
        selected_id = int(selected.split(":")[0])

    if st.button("Guardar"):
        if name.strip() and email.strip():
            bm.add_birthday(name.strip(), str(birthdate), email.strip(), selected_id)
            st.success("Cumpleaños guardado correctamente.")
        else:
            st.error("Por favor, completa todos los campos.")

elif choice == "Ver Próximos Cumpleaños":
    st.subheader("Cumpleaños Próximos")
    birthdays = bm.get_upcoming_birthdays()
    if not birthdays:
        st.info("No hay cumpleaños registrados.")
    for b in birthdays:
        st.write(f"{b.name} - en {b.days_until_birthday()} días")

elif choice == "Gestionar Mensajes":
    st.subheader("Mensajes de Felicitación")
    new_msg = st.text_area("Nuevo mensaje")
    if st.button("Agregar mensaje"):
        if new_msg.strip():
            mm.add_message(new_msg.strip())
            st.success("Mensaje agregado.")
        else:
            st.error("El mensaje no puede estar vacío.")

    for m in mm.messages:
        st.write(f"{m.id}: {m.text}")
        # Aquí puedes implementar editar o borrar mensajes si quieres

elif choice == "Enviar Felicitaciones Hoy":
    st.subheader("Enviar Correos de Felicitación")
    from_email = st.text_input("Correo emisor (tu email)")
    from_password = st.text_input("Contraseña del correo emisor", type="password")
    if st.button("Enviar felicitaciones"):
        enviar_felicitaciones(bm, mm, from_email, from_password)
        st.success("Intento de envío realizado. Revisa la consola para detalles.")


Overwriting app.py


In [47]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 23 packages in 790ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼

In [48]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.138.204.200
⠙your url is: https://tough-cows-bow.loca.lt
